# LangChain ReAct Agent with Couchbase via Model Context Protocol (MCP) - A Tutorial

This notebook demonstrates how to build a ReAct (Reasoning and Acting) agent using [LangChain](https://www.langchain.com/) and [LangGraph](https://www.langchain.com/langgraph) that can interact with a Couchbase database. The key to this interaction is the Model Context Protocol (MCP), which allows the AI agent to seamlessly connect to and use Couchbase as a tool. Read more about LangGraph's ReAct agent [here](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent).

## What is the Model Context Protocol (MCP)?

The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) is an open standard designed to standardize how AI assistants and applications connect to and interact with external data sources, tools, and systems. Think of MCP as a universal adapter that allows AI models to seamlessly access the context they need to produce more relevant, accurate, and actionable responses.

**Key Goals and Features of MCP:**

*   **Standardized Communication:** MCP provides a common language and structure for AI models to communicate with diverse backend systems, replacing the need for numerous custom integrations.
*   **Enhanced Context Management:** It helps manage the limited context windows of LLMs efficiently, enabling them to maintain longer, more coherent interactions and leverage historical data.
*   **Secure Data Access:** MCP emphasizes secure connections, allowing developers to expose data through MCP servers while maintaining control over their infrastructure.
*   **Tool Use and Actionability:** It enables LLMs to not just retrieve information but also to use external tools and trigger actions in other systems.
*   **Interoperability:** Fosters an ecosystem where different AI tools, models, and data sources can work together more cohesively.

MCP aims to break down data silos, making it easier for AI to integrate with real-world applications and enterprise systems, leading to more powerful and context-aware AI solutions.

**MCP Typically Follows a Client-Server Architecture:**
*   **MCP Hosts/Clients:** Applications (like AI assistants, IDEs, or other AI-powered tools) that want to access data or capabilities. In this demo, this notebook, through LangChain, acts as an MCP client.
*   **MCP Servers:** Lightweight programs that expose specific data sources or tools (e.g., a database, an API) through the standardized MCP. The `mcp-server-couchbase` project fulfills this role for Couchbase.


# Before you start
## Get Credentials for OpenAI
Please follow the [instructions](https://platform.openai.com/docs/quickstart) to generate the OpenAI credentials.
## Create and Deploy Your Free Tier Operational cluster on Capella

To get started with Couchbase Capella, create an account and use it to deploy a forever free tier operational cluster. This account provides you with an environment where you can explore and learn about Capella with no time constraint.

To learn more, please follow the [instructions](https://docs.couchbase.com/cloud/get-started/create-account.html).

### Couchbase Capella Configuration

When running Couchbase using [Capella](https://cloud.couchbase.com/sign-in), the following prerequisites need to be met.

* Create the [database credentials](https://docs.couchbase.com/cloud/clusters/manage-database-users.html) to access the required bucket (Read and Write) used in the application.
* [Allow access](https://docs.couchbase.com/cloud/clusters/allow-ip-address.html) to the Cluster from the IP on which the application is running.
* Your Capella free-tier account includes a travel-sample bucket, with sample documents used for booking and travel purposes. You can find more information [here](https://docs.couchbase.com/cloud/get-started/run-first-queries.html).

## Setup Instructions

Before running this notebook, ensure you have the following prerequisites met:

*   **`mcp-server-couchbase` Project:**
    *   Clone the `mcp-server-couchbase` project from [here](https://github.com/Couchbase-Ecosystem/mcp-server-couchbase).
    *   You'll need the local path to this project to start the MCP server from this notebook. **Update this path in the `StdioServerParameters` cell later if yours is different.**
*   **Set Environment Variables:** This notebook loads the OpenAI API key and other environment variables from the `.env` file. Include the following:

    ```
    OPENAI_API_KEY=your_openai_api_key_here
    CB_CONNECTION_STRING=your_couchbase_connection_string
    CB_USERNAME=your_couchbase_username
    CB_PASSWORD=your_couchbase_password
    CB_BUCKET_NAME=your_target_bucket # e.g., travel-sample
    ```

    We have already included a `.env.sample` file. Change the file name to `.env` and fill in the environment variables.
*   **Setup uv:** uv is a modern and fast python package and project manager. We will use uv to run the MCP server. Install uv from [here](https://docs.astral.sh/uv/getting-started/installation/#installing-uv).
*   **Python Libraries:** Install the necessary libraries by running the code cell below.

In [1]:
!pip install -q 'langchain==0.3.25' 'langgraph==0.4.0' 'langchain-openai==0.3.14' 'langchain-mcp-adapters==0.0.9' 'python-dotenv==1.1.0'

## Importing Necessary Libraries

This cell imports the essential Python tools for our project:

*   **`dotenv` & `os`:** For loading and using secret API keys and other settings from a `.env` file.
*   **`mcp` (ClientSession, StdioServerParameters, stdio_client):** For connecting this notebook (as a client) to the MCP server, which in turn talks to Couchbase.
*   **`langchain_mcp_adapters.tools` (`load_mcp_tools`):** To make the Couchbase tools (exposed via MCP) usable by our LangChain AI agent.
*   **`langgraph.prebuilt` (`create_react_agent`):** To easily build a "ReAct" AI agent that can think and use tools.
*   **`langgraph.checkpoint.memory` (`InMemorySaver`):** To help the agent remember past parts of the conversation.
*   **`langchain_openai` (`ChatOpenAI`):** To connect to and use OpenAI's language models (like GPT-4).

Running this cell makes all these components ready to use.

In [2]:
from dotenv import load_dotenv
import os

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI

load_dotenv()

True

## Defining the Question-Answering Function

This cell defines an asynchronous function `qna(agent)` that we'll use to interact with our ReAct agent.

*   It takes the created `agent` as an argument.
*   `config = {"configurable": {"thread_id": "1"}}`: This configuration is important for LangGraph agents. It uses a `thread_id` to maintain conversation state. Using the same `thread_id` across multiple calls to the agent allows it to remember previous interactions in that "thread."
*   The function then defines a series of example questions (`message`) which we want to ask the agent. The agent queries the Couchbase MCP to get travel related data, formats it and presents it to the user.
*   This function allows us to easily test the agent with multiple queries in sequence.

In [3]:
async def qna(agent):
    config = {"configurable": {"thread_id": "1"}}

    message = "Tell me about the database that you are connected to."
    print(f"\n\n**Running:** {message}\n")
    result = await agent.ainvoke({"messages": message}, config)
    print(result["messages"][-1].content)
    print('-'*50)

    message = "List out the top 5 hotels by the highest aggregate rating?"
    print(f"\n\n**Running**: {message}\n")
    result = await agent.ainvoke({"messages": message}, config)
    print(result["messages"][-1].content)
    print('-'*50)

    message = "Recommend me a flight and hotel from New York to San Francisco"
    print(f"\n\n**Running**: {message}\n")
    result = await agent.ainvoke({"messages": message}, config)
    print(result["messages"][-1].content)
    print('-'*50)

    message = "I'm going to the UK for 1 week. Recommend some great spots to visit for sightseeing. Also mention the respective prices of those places for adults and kids."
    print(f"\n\n**Running**: {message}\n")
    result = await agent.ainvoke({"messages": message}, config)
    print(result["messages"][-1].content)
    print('-'*50)

    message = "My budget is around 30 pounds a night. What will be the best hotel to stay in?"
    print(f"\n\n**Running**: {message}\n")
    result = await agent.ainvoke({"messages": message}, config)
    print(result["messages"][-1].content)
    print('-'*50)

## Defining the System Prompt

The system prompt is a crucial piece of instruction given to the Large Language Model (LLM) that powers our agent. It sets the context, defines the agent's persona, capabilities, and constraints.

In this system prompt:
*   We explain the **Couchbase data hierarchy** (Cluster, Bucket, Scope, Collection, Document) to help the LLM understand how the data is organized.
*   We specifically instruct the agent that **"The data is inside `inventory` scope, so use only that scope."** This focuses the agent on the relevant part of the `travel-sample` database.
*   We provide **SQL++ query generation guidelines**:
    *   "Any query you generate needs to have only the collection name in the FROM clause."
    *   "Every field, collection, scope or bucket name inside the query should be inside backticks."
*   The overall goal is to guide the LLM to use the provided MCP tools (which will be Couchbase operations) effectively and to formulate correct SQL++ queries for the `inventory` scope.

A well-crafted system prompt significantly improves the agent's performance and reliability.

In [4]:
system_prompt = """Couchbase organizes data with the following hierarchy (from top to bottom):

                1. Cluster

                The overall container of all Couchbase data and services.

                2. Bucket

                A bucket is similar to a database in traditional systems.

                Each bucket contains multiple scopes.

                Example: "users", "analytics", "products"

                3. Scope

                A scope is a namespace within a bucket that groups collections.

                Scopes help isolate data for different microservices or tenants.

                Default scope name: _default

                4. Collection

                The equivalent of a table in relational databases.

                Collections store JSON documents.

                Default collection name: _default

                5. Document

                The atomic data unit (usually JSON) stored in a collection.

                Each document has a unique key within its collection.

                Use the tools to read the database answer questions based on this database.
                The data is inside `inventory` scope, so use only that scope.
                Any query you generate needs to have only the collection name in the FROM clause.
                Every field, collection, scope or bucket name inside the query should be inside backticks."""

## Configuring the Language Model and MCP Server

This cell sets up two key components:

1.  **`model = ChatOpenAI(model="gpt-4.1")`**: This line initializes the LLM we'll be using. We're choosing `gpt-4.1` from OpenAI.
2.  **`server_params = StdioServerParameters(...)`**:
    *   This configures how our Python script will start and communicate with the `mcp-server-couchbase` application.
    *   `command="uv"` and `args=[...]`: This specifies the command to run. Here, it's using `uv run` (a fast Python project and virtual environment manager) to execute the `mcp_server.py` script located in the `mcp-server-couchbase` project directory.
        *   **IMPORTANT:** You **MUST** update the `"--directory"`, path to point to the correct location of your cloned `mcp-server-couchbase` repository.
    *   `env={...}`: This dictionary defines environment variables that will be passed to the MCP server process when it starts. These are crucial for the MCP server to connect to your Couchbase instance:

In [5]:
model = ChatOpenAI(model="gpt-4.1")

server_params = StdioServerParameters(
    command="uv",
    args=[
        "--directory",
        "/path/to/mcp-server-couchbase",
        "run",
        "src/mcp_server.py"
    ],
    env={
        "CB_CONNECTION_STRING": os.getenv("CB_CONNECTION_STRING"),
        "CB_USERNAME": os.getenv("CB_USERNAME"),
        "CB_PASSWORD": os.getenv("CB_PASSWORD"),
        "CB_BUCKET_NAME": os.getenv("CB_BUCKET_NAME")
    }
)

## Defining the Main Execution Logic

The `main` function ties everything together to set up and run our agent:

1.  **Start & Connect to MCP Server:** It first starts the `mcp-server-couchbase` process using `stdio_client` and establishes a communication `ClientSession` with it.
2.  **Initialize Session & Load Tools:** The MCP `session` is initialized. Then, `load_mcp_tools` queries the MCP server to get the available Couchbase tools and prepares them for LangChain.
3.  **Set Up Agent Memory:** `InMemorySaver` is created to allow the agent to remember conversation history.
4.  **Create ReAct Agent:** The `create_react_agent` function builds our AI agent, providing it with the language `model`, the Couchbase `tools`, our `system_prompt`, and the `checkpoint` for memory.
5.  **Run Q&A:** Finally, it calls the `qna` function, passing the created `agent` to start the question-and-answer process with the database.

In [6]:
async def main():
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            print("Initializing connection...")
            await session.initialize()

            # Get tools
            print("Loading tools...")
            tools = await load_mcp_tools(session)

            # Create and run the agent
            print("Creating agent...")
            checkpoint = InMemorySaver()

            agent = create_react_agent(
                model, 
                tools,
                prompt=system_prompt,
                checkpointer=checkpoint
            )

            print("-"*25, "Starting Run", "-"*25)
            await qna(agent)

## Running the Agent

This final cell simply executes the `await main()` function.

When you run this cell:
1.  The `mcp-server-couchbase` process will be started in the background.
2.  The Python script will connect to it as an MCP client.
3.  The LangChain ReAct agent will be initialized with the Couchbase tools exposed via MCP.
4.  The agent will then attempt to answer the series of questions defined in the `qna` function by:
    *   Reasoning about the question.
    *   Deciding if a Couchbase tool is needed.
    *   Formulating a SQL++ query (if appropriate, based on the system prompt).
    *   Executing the tool (which sends the query to the MCP server, which then runs it on Couchbase).
    *   Using the tool's output to generate a natural language response.

You will see the questions and the agent's answers printed below. This demonstrates the end-to-end flow of a natural language query being translated into database actions and then into a user-friendly response, all orchestrated by the LangChain agent using MCP.

In [7]:
await main()

Initializing connection...
Loading tools...
Creating agent...
------------------------- Starting Run -------------------------


**Running:** Tell me about the database that you are connected to.

I am connected to a Couchbase database that contains several scopes and collections. The main scope relevant for queries is inventory. Within the inventory scope, there are the following collections:

- cache
- route
- landmark
- hotel
- airport
- airline

Each collection contains documents (typically in JSON format), and you can query or retrieve data from these collections as needed. The structure is as follows:

Cluster → Bucket → Scope (inventory) → Collection (cache, route, landmark, hotel, airport, airline) → Document

Let me know if you want details about any specific collection or want to see the data or schema from any of them.
--------------------------------------------------


**Running**: List out the top 5 hotels by the highest aggregate rating?

The top 5 hotels by highest aggr